1. 문서의 내용을 읽는다. 
2. 문서를 쪼갠다. 
    - 토큰 수 초과로 답변을 생성하지 못할 수 있고 
    - 문서가 길면 (input 이 길면) 답변 생성이 오래걸림 
3. 임베딩 -> 벡터 데이터베이스에 저장 
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색 
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 

In [1]:
%pip install --upgrade --quiet docx2txt langchain-community


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader("./tax.docx")
document = loader.load()

In [3]:
document
len(document)

1

In [4]:
%pip install -qU langchain-text-splitter

ERROR: Could not find a version that satisfies the requirement langchain-text-splitter (from versions: none)
ERROR: No matching distribution found for langchain-text-splitter

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [34]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)

loader = Docx2txtLoader("./tax.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)


In [35]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")


In [ ]:
%pip install --upgrade langchain-openai langchain-chroma openai

In [37]:
from dotenv import load_dotenv
from langchain_chroma import Chroma

#database = Chroma.from_documents(documents=document_list, embedding=embeddings, collection_name='chroma-tax', persist_directory="./chroma")
database = Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embeddings)

In [38]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
#retrieved_docs = database.similarity_search(query, k=3)

In [13]:
retrieved_docs

[Document(id='e937767d-b54c-4867-84ab-2cc208f10ea3', metadata={'source': './tax.docx'}, page_content='[전문개정 2009. 12. 31.]\n\n\n\n제10조(납세지의 변경신고) 거주자나 비거주자는 제6조부터 제9조까지의 규정에 따른 납세지가 변경된 경우 변경된 날부터 15일 이내에 대통령령으로 정하는 바에 따라 그 변경 후의 납세지 관할 세무서장에게 신고하여야 한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2장 거주자의 종합소득 및 퇴직소득에 대한 납세의무 <개정 2009. 12. 31.>\n\n\n\n제1절 비과세 <개정 2009. 12. 31.>\n\n\n\n제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31.>\n\n1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외

In [39]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [24]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다. 
- [Context]를 참고해서 사용자의 질문에 답변해주세요

[Context]
{retrieved_docs}

Question: {query}
"""

In [25]:
ai_message = llm.invoke(prompt)

In [26]:
ai_message.content

'한국 소득세는 개인의 연간 소득에 따라 계산되며, 기본적으로 과세표준 구간과 세율에 따라 결정됩니다. 연봉 5천만원인 직장인의 소득세를 계산하려면 다음 단계를 따릅니다:\n\n1. **총소득에서 공제**: 우선, 연봉에서 근로소득공제를 적용하여 과세표준을 계산해야 합니다. 2023년 기준으로 근로소득공제는 다음과 같습니다:\n   - 5백만원 이하: 전액\n   - 5백만원 초과 ~ 1,500만원 이하: 소득의 70% + 150만원\n   - 1,500만원 초과 ~ 4,500만원 이하: 소득의 40% + 750만원\n   - 4,500만원 초과 ~ 1억원 이하: 소득의 15% + 1,950만원\n   - 1억원 초과: 1,950만원 공제\n\n   연봉 5천만원의 경우, 공제액은 5천만원의 15% + 1,950만원 = 750만원입니다. 따라서 과세표준은 5천만원 - 750만원 = 4,250만원입니다.\n\n2. **소득세 계산**: 과세표준에 따라 소득세를 계산합니다. 2023년 기준 소득세율은 다음과 같습니다:\n   - 1,200만원 이하: 6%\n   - 1,200만원 초과 ~ 4,600만원 이하: 15%\n   - 4,600만원 초과 ~ 8,800만원 이하: 24%\n   - 8,800만원 초과 ~ 1억5천만원 이하: 35%\n   - 1억5천만원 초과: 38%\n\n   4,250만원은 1,200만원 초과 ~ 4,600만원 이하 구간에 해당하므로:\n   - 1,200만원까지는 6%: 1,200만원 x 0.06 = 72만원\n   - 1,200만원 초과분(4,250만원 - 1,200만원)에는 15%: 3,050만원 x 0.15 = 457.5만원\n\n   총 소득세는 72만원 + 457.5만원 = 529.5만원입니다.\n\n3. **세액공제 적용**: 최종적으로 세액공제를 적용하여 정확한 소득세를 결정합니다. 세액공제는 다양한 항목에 따라 다르므로 개인의 상황에 맞게 적용해야 합니다.\n\n이 계산은 기본적인 소득세 계산 단계이며, 개인의 경우 납세

In [27]:
%pip install -U langchain langchainhub --quiet


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [40]:
from langchain import hub 

prompt = hub.pull("rlm/rag-prompt")

/Users/suuuheee/.pyenv/versions/llm-application/lib/python3.10/site-packages/langsmith/client.py:288: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [29]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [41]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [42]:
ai_message = qa_chain({"query": query})

In [43]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '제공된 문서에는 연봉 5천만원인 직장인의 소득세에 대한 구체적인 계산 정보가 포함되어 있지 않습니다. 따라서 소득세를 정확히 계산하려면 소득세율, 공제 항목 등 추가적인 세법 정보가 필요합니다. 이러한 정보는 국세청에서 제공되는 소득세율 표를 통해 확인할 수 있습니다.'}